In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
from preprocessy.pipelines import StandardPipeline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
def test_input():
    params = {'target_label':"Survived",'cat_cols':['Pclass','Sex','Parch','Embarked'],'drop_cols':["PassengerId","Name","Ticket","Cabin"],"fill_missing":{"mean":["Age"],"median":["Fare"]}}
    p = StandardPipeline(train_df_path="../datasets/titanic.csv",test_df_path="../datasets/titanic.csv", params=params)
    p.save_config('../datasets/configs/config.json')
    p.process()
    
    return params
    
    

In [5]:
params = test_input()


Pipeline Class: StandardPipeline

+-----------------------+---------------------------------------------------------------------------------------+
| Pipeline Property     | Value                                                                                 |
+-----------------------+---------------------------------------------------------------------------------------+
| Train Dataframe Path  | ../datasets/titanic.csv                                                               |
| Test Dataframe Path   | ../datasets/titanic.csv                                                               |
| Config File Path      | None                                                                                  |
| Pipeline Stages       | Read file, Parse dataset, Execute, Encode, Handle outliers, Execute, Train test split |
| Total Pipeline Stages | 7                                                                                     |
| Total Params          | 6                          

In [6]:
params['train_df']

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,PclassEncoded,SexEncoded,ParchEncoded,EmbarkedEncoded
0,0,3,male,22.000000,1,0,7.2500,S,0,0,0,0
1,1,1,female,38.000000,1,0,71.2833,C,1,1,0,1
3,1,1,female,35.000000,1,0,53.1000,S,1,1,0,0
9,1,2,female,14.000000,1,0,30.0708,C,2,1,0,1
13,0,3,male,39.000000,1,5,31.2750,S,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...
861,0,2,male,21.000000,1,0,11.5000,S,2,0,0,0
866,1,2,female,27.000000,1,0,13.8583,C,2,1,0,1
871,1,1,female,47.000000,1,1,52.5542,S,1,1,1,0
874,1,2,female,28.000000,1,0,24.0000,C,2,1,0,1


In [7]:
train_new = params["X_train"]
test_new = params["X_test"]
train_y = params["y_train"]
test_y = params["y_test"]
colstodrop = ["Sex","Parch","PclassEncoded","Embarked"]
train_new.drop(colstodrop,axis=1,inplace=True)
test_new.drop(colstodrop,axis=1,inplace=True)

In [9]:
original = pd.read_csv("../datasets/titanic.csv")
cols = ["PassengerId","Name","Ticket","Cabin"]
original.drop(cols,axis=1,inplace=True)
original.dropna(inplace=True)
original["SexEncoded"] = original["Sex"].apply(lambda x: 0 if x=='female' else 1)
original["EmbarkedEncoded"] = original["Embarked"].apply(lambda x: 0 if x=='C' else( 1 if x=='S' else 2))
original.drop(['Sex','Embarked'],axis=1,inplace=True)
y = original["Survived"]
X = original.drop(["Survived"],axis=1)
trainx, testx, trainy, testy = train_test_split(X,y,test_size=0.2)

In [10]:
l = LogisticRegression(max_iter=200)
l.fit(train_new,train_y)

LogisticRegression(max_iter=200)

In [11]:
print("model score: %.3f" % l.score(test_new, test_y))

model score: 0.813


In [12]:
def sc(X,y):
    trainx, testx, trainy, testy = train_test_split(X,y,test_size=0.2)
    l = LogisticRegression(max_iter=200)
    l.fit(trainx,trainy)
    t = l.score(testx, testy)
    return t

In [13]:
z = []
for _ in range(5):
    z.append(sc(X,y))
sum(z)/len(z)


0.8223776223776224

In [14]:
z

[0.7902097902097902,
 0.7972027972027972,
 0.8251748251748252,
 0.8461538461538461,
 0.8531468531468531]